In [190]:
import sqlalchemy
import pandas as pd
import pandasql as ps

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct, and_, inspect

import datetime as dt
import requests
import math
import numpy as np


In [100]:

engine = create_engine("sqlite:///Earthquake.db")
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()


[]

In [8]:

conn = engine.connect()
quakes_pd = pd.read_sql("SELECT * FROM quakes", conn)


    

,e_id,date_time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,0,1975-12-31 14:34:22.080000000,36.813500,-121.132333,5.881,2.71,md,50.0,97.00,0.06757,...,2016-12-15T05:09:40.840Z,Central California,earthquake,0.23,0.640,0.09,15.0,reviewed,nc,nc
1,1,1975-12-31 02:53:56.256000000,34.279000,-116.337000,2.330,2.70,md,0.0,71.00,None,...,2016-01-29T21:39:32.720Z,"16km N of Joshua Tree, CA",earthquake,None,0.847,None,0.0,reviewed,ci,ci
2,2,1975-12-30 23:37:52.896000000,36.905667,-121.637167,-0.155,2.67,md,13.0,137.00,0.03243,...,2016-12-15T05:08:54.636Z,Central California,quarry blast,0.34,1.060,0.77,3.0,reviewed,nc,nc
3,3,1975-12-30 23:04:47.424000000,36.838667,-121.569833,4.774,2.67,md,46.0,81.00,0.01892,...,2016-12-15T05:08:50.998Z,Central California,earthquake,0.21,0.360,0.28,11.0,reviewed,nc,nc
4,4,1975-12-30 18:10:28.128000000,35.781000,-121.421000,11.895,3.15,md,17.0,247.00,0.2396,...,2016-12-15T05:08:39.866Z,offshore Central California,earthquake,1.04,0.610,0.55,4.0,reviewed,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80382,80382,2016-01-03 09:12:46.368000000,41.848300,-119.601600,6.700,2.50,ml,8.0,75.42,0.179,...,2018-07-02T17:44:50.027Z,"72km ESE of Lakeview, Oregon",earthquake,None,7.200,0.34,5.0,reviewed,nn,nn
80383,80383,2016-01-03 04:34:59.808000000,41.869000,-119.613700,9.900,2.50,ml,8.0,99.14,0.159,...,2018-07-02T17:44:48.976Z,"70km ESE of Lakeview, Oregon",earthquake,None,4.200,0.21,5.0,reviewed,nn,nn
80384,80384,2016-01-02 06:38:32.928000000,38.085667,-121.916000,20.430,2.50,md,68.0,83.00,0.0982,...,2017-02-09T18:12:19.290Z,"6km NNW of Pittsburg, California",earthquake,0.26,0.190,0.155,71.0,reviewed,nc,nc
80385,80385,2016-01-01 20:02:39.552000000,40.175500,-122.228833,32.750,2.56,md,20.0,124.00,0.08559,...,2017-02-09T18:04:27.831Z,"0km ESE of Red Bluff, California",earthquake,0.83,1.370,0.154,47.0,reviewed,nc,nc


In [16]:
lon = -120
lat = 36
radius = 30
start_date = '2016-01-01'
end_date = '2017-05-05'

cols = 'date_time, latitude, longitude, depth, mag'
sql = f"SELECT * FROM quakes where {start_date} < date_time and date_time < {end_date}"
quakes_pd = pd.read_sql(sql, conn)

quakes_pd


,e_id,date_time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


## Retrieve last 30 days function

In [191]:


def ReadUSGS(BaseDate = dt.datetime.today()):
    
    StartDate = (BaseDate - dt.timedelta(days=30)).strftime('%Y-%m-%d %H:%M:%S')
    EndDate = BaseDate.strftime('%Y-%m-%d %H:%M:%S')
    
    url = f'https://earthquake.usgs.gov/fdsnws/event/1/query.geojson?starttime={StartDate}&endtime={EndDate}&maxlatitude=41.961&minlatitude=32.813&maxlongitude=-114.521&minlongitude=-124.255&minmagnitude=2.5&orderby=time'
    
    data = requests.get(url)
    return data.json()

In [192]:
def AddPOR(Main_df):
    periods = np.array([])
    for index, row in Main_df.iterrows():
        catagory = f"Mag{int(row['category']*10)}"
        city = row['ClosestCity']
        periods = np.append(periods, POR_df.loc[city,catagory])

    Main_df['POR'] = periods

    return Main_df

In [193]:

citygeos = pd.read_csv("city geos.csv")



In [194]:

def FindCity(lat, lon):
    distCompare = 100000000
    for i in range(0,len(citygeos)):
        latCity = citygeos.iloc[i,1]
        lonCity = citygeos.iloc[i,2]
        x = latCity - lat
        y = (lonCity - lon) * math.cos(lat * math.pi/180)
        distance = 110.25 * (x**2 +y**2)**0.5
        if distance < distCompare:
            Geo = citygeos.iloc[i,0]
            distCompare = distance
    return Geo


In [195]:
def df_to_json(df):
    
    features = []
    
    for index, row in df.iterrows():
        
        # Read record from dataframe 
        lon = row['longitude']
        lat = row['latitude']
        depth = row['depth']
        mag = row['mag']
        city = row['ClosestCity']
        
        # build record into dictionary 
        record = {
            
            'geometry':{
                'coordinates': [lon, lat, depth]
            },
            
            "properties": {
                'mag': mag,
                'title':city
            }
              
        }
                   
        features.append(record)

    return {'features':features}
        
        
    

In [196]:
POR_df = pd.read_csv('POR.csv')
POR_df.set_index('ClosestCity',inplace = True)
POR_df

,Mag20,Mag25,Mag30,Mag35,Mag40,Mag45,Mag50,Mag55,Mag60,Mag65,Mag70,Mag75,Mag80,Mag85,Mag90
ClosestCity,,,,,,,,,,,,,,,
Anaheim,2500.0,0.549451,2.173913,5.000000,16.666667,50.000000,50.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Antelope Peak NV,2500.0,2.083333,3.333333,3.571429,16.666667,16.666667,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Avenal CA,2500.0,0.027579,0.067204,0.196078,0.581395,2.272727,7.142857,16.666667,2500.0,50.0,2500.0,2500.0,2500.0,2500.0,2500.0
Azusa,2500.0,0.649351,3.125000,7.142857,25.000000,2500.000000,2500.000000,50.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Bakersfield,2500.0,0.085911,0.314465,1.250000,2.941176,7.142857,16.666667,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Walnut Creek,2500.0,0.213675,0.641026,2.500000,5.555556,2500.000000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Warm Springs NV,2500.0,0.500000,1.351351,3.846154,10.000000,2500.000000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
"Willits, CA",2500.0,0.109409,0.304878,1.666667,3.333333,12.500000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0


In [197]:

def DefineProblem(AtDate):
    
    AtDate ='2020-10-17 00:00:00'
    myDate = dt.datetime.strptime(AtDate, '%Y-%m-%d %H:%M:%S')

    response = ReadUSGS(myDate)
    rows = []

    for record in response['features']:

        mag = record['properties']['mag']
        lon = record['geometry']['coordinates'][0]
        lat = record['geometry']['coordinates'][1]

        if mag - int(mag) > .5:
            n = 0.5 
        else:
            n = 0


        row = {

            'longitude': lon,
            'latitude': lat,
            'depth': record['geometry']['coordinates'][2],
            'mag': mag,
            'category': int(mag) + n,
            'ClosestCity': FindCity(lat=lat,lon=lon)

        }

        rows.append(row)

    df=pd.DataFrame(rows)

    df = AddPOR(df)

    df
    
    
    
    






In [198]:
# for item in df.groupby('ClosestCity')['category'].value_counts():
#
# problem_df[problem_df['category'] == 2.5].count()
# [df['category'] == 2.5]
# df[df['category'] == 2.5].count()
agg = df.groupby('ClosestCity')['category'].value_counts()
# for item in agg.index:
    
#     print(item, agg[item[0]][item[1]])
cities = []
for item in agg.index:
    print(item)


('Avenal CA', 2.5)
('Bishop', 2.5)
('Bishop', 3.0)
('El Centro CA', 2.5)
('El Centro CA', 3.0)
('El Centro CA', 3.5)
('El Centro CA', 4.0)
('El Centro CA', 2.0)
('El Centro CA', 4.5)
('Eureka CA', 2.5)
('Eureka CA', 3.0)
('Fremont', 2.5)
('Fremont', 3.0)
('Hawthorne NV', 2.5)
('Hawthorne NV', 2.0)
('Hawthorne NV', 3.0)
('Hawthorne NV', 3.5)
('Hayfork Ca', 3.5)
('Hemet', 2.5)
('King City NV', 2.5)
('King City NV', 3.0)
('Lone Pine', 2.5)
('Lone Pine', 3.0)
('Oakland', 2.5)
('Ontario', 2.5)
('Ontario', 3.0)
('Palm Springs', 2.5)
('Palm Springs', 3.0)
('Palm Springs', 3.5)
('Pasadena', 4.5)
('Reno', 2.5)
('Ridgecrest CA', 2.5)
('Ridgecrest CA', 3.0)
('Riverside', 3.0)
('Round Mtn NV', 2.0)
('Salinas', 2.5)
('Salinas', 3.0)
('San Bernardino', 2.5)
('San Bernardino', 3.0)
('San Jose', 2.5)
('Twenty Nine Palms', 2.5)
('Ukiah, CA', 2.5)
('Walnut Creek', 2.5)
('Willits, CA', 2.5)
('Willits, CA', 3.0)


In [199]:
df

,longitude,latitude,depth,mag,category,ClosestCity,POR
0,-119.597900,39.197000,7.70,2.70,2.5,Reno,0.120773
1,-121.634500,36.977000,3.75,2.52,2.5,Salinas,0.020145
2,-124.089833,40.940500,27.98,2.65,2.5,Eureka CA,0.065445
3,-117.875700,38.162200,8.00,2.70,2.5,Hawthorne NV,0.032765
4,-121.536667,36.029333,8.01,2.98,2.5,King City NV,0.020833
...,...,...,...,...,...,...,...
264,-117.992167,36.442167,3.73,3.07,3.0,Lone Pine,0.210970
265,-121.275167,36.653667,3.92,2.84,2.5,Salinas,0.020145
266,-121.276667,36.654833,3.81,2.66,2.5,Salinas,0.020145
267,-123.365167,39.854833,6.95,2.62,2.5,"Willits, CA",0.109409


In [211]:
features = POR_df.columns

In [212]:
cities =df['ClosestCity'].unique()

In [189]:
pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26822 sha256=14496400524c4ad756c7711d98dd97f232f6288ad2f91a55845b6469cbe4e9ce
  Stored in directory: c:\users\spaki\appdata\local\pip\cache\wheels\5c\4b\ec\41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.
